In [11]:
%gui qt
from mayavi import mlab
import numpy as np
import time
import os


In [12]:
# for testing
mlab.test_contour3d_anim()

In [25]:
DATA_SET = "data_multi_2"


Now, the numpy array is num_frames x 3 x number of satellites

In [14]:
# # Tmp code to mix all three bodies
# data0 = data = np.load(f"data/data_0.npy")
# data1 = data = np.load(f"data/data_1.npy")
# data2 = data = np.load(f"data/data_2.npy")

# print(data0.shape)
# print(data1.shape)
# print(data2.shape)

(525600, 3)
(525600, 3)
(525600, 3)


In [26]:

# DATA_SCALE_FACTOR = radius_earth
DATA_SCALE_FACTOR = 6.378e6

# Lets divide everything by R, to get intution
# new_shape = list(data0.shape)
# new_shape.append(3)

data = np.load(f"data/{DATA_SET}.npy")

# data[:, : , 0] = data0
# data[:, : , 1] = data1
# data[:, : , 2] = data2
data = data/DATA_SCALE_FACTOR


print(data.shape)
print(data[:10])

(525600, 3, 2)
[[[-1.10490974 -1.09189444]
  [ 1.16406394  1.37920929]
  [ 1.3131193   1.02106045]]

 [[-1.1534507  -1.15574597]
  [ 1.20302208  1.37829771]
  [ 1.35197449  1.02039622]]

 [[-1.20130704 -1.21888054]
  [ 1.24125893  1.37648051]
  [ 1.39001604  1.01906155]]

 [[-1.24851858 -1.28129308]
  [ 1.27882328  1.37380224]
  [ 1.42730182  1.0170894 ]]

 [[-1.29512116 -1.3429806 ]
  [ 1.31575842  1.37030522]
  [ 1.46388298  1.01451109]]

 [[-1.34114722 -1.40394194]
  [ 1.35210304  1.36602958]
  [ 1.49980506  1.01135634]]

 [[-1.3866262  -1.46417749]
  [ 1.38789187  1.36101338]
  [ 1.53510878  1.00765332]]

 [[-1.43158499 -1.52368902]
  [ 1.42315625  1.35529272]
  [ 1.56983076  1.00342873]]

 [[-1.47604816 -1.58247946]
  [ 1.45792456  1.34890177]
  [ 1.60400406  0.99870789]]

 [[-1.52003831 -1.64055277]
  [ 1.49222264  1.34187295]
  [ 1.63765866  0.99351477]]]


In [27]:
# Frame of reference setup
frame_x, frame_y, frame_z = np.mgrid[-10: 10: 2j,-10: 10: 2j,-10: 10: 2j]

mlab.clf()

# Uncomment the below line to make endpoints invisible
# mlab.points3d(frame_x, frame_y, frame_z, opacity=0)
mlab.points3d(frame_x, frame_y, frame_z, opacity=0.0)


In [28]:

# So as to get spatial visualization in mayavi
VISUAL_SCALE_FACTOR = 3

In [29]:
earth_obj = mlab.points3d(0,0,0, color = (1,0,0))


sat_init_pos = data[0] * VISUAL_SCALE_FACTOR
# size is 3 * num_satellites
print((sat_init_pos[0]))
print(sat_init_pos[1])
print(sat_init_pos[2])

sat_objs = mlab.points3d(
        sat_init_pos[0],
        sat_init_pos[1],
        sat_init_pos[2],
        # color = (0.5, 0.5, 0.5)
        scale_factor = 1
    )
    

# print(sat_init_pos)


sat_trajs =[
    mlab.plot3d(
        sat_init_pos[0, i],
        sat_init_pos[1, i],
        sat_init_pos[2, i],
    )

    for i in range(data.shape[2])

]
print(sat_trajs)


[-3.31472923 -3.27568332]
[3.49219183 4.13762787]
[3.93935789 3.06318134]
[<mayavi.modules.surface.Surface object at 0x0000022199B27360>, <mayavi.modules.surface.Surface object at 0x0000022199B27040>]


In [30]:
# Lets once try mlab animate decorator...
# If it doesn't work, do with save_fig and delay and all
# @mlab.animate

# Not using decorator here, as I need inputs to this function...

def anim(moving_obj, data, num_frames = 50, obj_trajs = None, pics_folder_name = None):

    # decorator not working... Animation stops in between
    # Now again,it doesn't stop

    delay = 100

    # @mlab.animate(delay = 100)

    if not os.path.exists(pics_folder_name):
                os.makedirs(pics_folder_name)
    if pics_folder_name is None:
         pics_folder_name = "pics"

    for i in range(num_frames):
        moving_obj.mlab_source.x = data[i][0] * VISUAL_SCALE_FACTOR  
        moving_obj.mlab_source.y = data[i][1] * VISUAL_SCALE_FACTOR 
        moving_obj.mlab_source.z = data[i][2] * VISUAL_SCALE_FACTOR 

        # Trajectory is not working, bodies just dont animate ...

        if obj_trajs is not None:
            for traj_idx, obj_traj in enumerate(obj_trajs):
                obj_traj.mlab_source.reset(
                    x = data[:i+1, 0, traj_idx] * VISUAL_SCALE_FACTOR,
                    y = data[:i+1, 1, traj_idx] * VISUAL_SCALE_FACTOR,
                    z = data[:i+1, 2, traj_idx] * VISUAL_SCALE_FACTOR
                )

        # Uncomment below line, if using decorator
        # yield
        # uncomment below lines, if not using decorator
        mlab.process_ui_events()
        # This savefig is a cruel trick to re-create the image or smthing like that
        mlab.savefig('./{}/anim_{:03}.png'.format(pics_folder_name, i))

        # time.sleep(delay/1000)



In [31]:
num_frames_interest = 200


anim(sat_objs, data, 
     num_frames = num_frames_interest, 
     obj_trajs = sat_trajs,
     pics_folder_name =  f"pics/{DATA_SET}/"
       )


# Obtained good animation results! Creates a folder with pic names for the same